In [4]:
import os
from google.colab import drive
MOUNTPOINT = '/content/drive'
drive.mount(MOUNTPOINT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
file_path = "/content/drive/MyDrive/My_project/house_price_prediction/"

In [53]:
!pip install --upgrade scikit-learn==1.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 28.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.0
    Uninstalling scikit-learn-1.3.0:
      Successfully uninstalled scikit-learn-1.3.0


In [7]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import pickle
import joblib



In [8]:
import sklearn

In [9]:
sklearn.__version__

'1.5.0'

In [10]:
house_data= pd.read_csv(file_path+"house_data2.csv", encoding='latin-1')
dallas_hpi = pd.read_csv(file_path+"Dallas_HPIF.csv", encoding='latin-1')
house_data['SOLD_MONTH'] = pd.to_datetime(house_data['SOLD_MONTH'], format='%m/%d/%Y').dt.date
dallas_hpi['DATE'] = pd.to_datetime(dallas_hpi['DATE'], format='%m/%d/%Y').dt.date

In [11]:

# house_data['SOLD_MONTH'] = pd.to_datetime(house_data['SOLD_MONTH'], format='%m/%d/%Y').dt.date

In [8]:
house_data.head()

,SOLD_MONTH,CITY,ZIP,PRICE,BEDS,BATHS,SQUAREFEET,LOTSIZE,YEARBUILT,Price_per_sqrt,MLS,LATITUDE,LONGITUDE,HOUSE_DES
0,2022-11-01,Carrollton,75007,375000,3.0,2.0,1821.0,8059.0,1974.0,206.0,20193171,33.000565,-96.880989,Beautifully updated and well-maintained home o...
1,2022-04-01,Carrollton,75007,389900,4.0,2.0,1706.0,7841.0,1979.0,229.0,20006334,33.008769,-96.872734,Lovely Carrollton gem with sparkling POOL! Ori...
2,2022-09-01,Carrollton,75007,608000,3.0,3.0,2721.0,9191.0,1986.0,223.0,20131908,32.989801,-96.915337,Beautifully updated home with a spectacular pa...
3,2022-09-01,Carrollton,75007,410000,3.0,2.0,1822.0,7492.0,1984.0,225.0,20141065,32.991635,-96.894394,Multiple offers received. Please submit highes...
4,2022-08-01,Carrollton,75007,598000,5.0,3.5,3141.0,9845.0,1989.0,190.0,20097778,33.003826,-96.918628,Beautiful 5 bed charmer with a POOL in Carroll...


In [9]:
dallas_hpi.head()

,DATE,HPI
0,2000-01-01,100.00000
1,2000-02-01,101.82326
2,2000-03-01,102.03669
3,2000-04-01,103.20051
4,2000-05-01,104.11440


In [ ]:
# dallas_hpi['DATE'] = pd.to_datetime(dallas_hpi['DATE'], format='%m/%d/%Y').dt.date

In [10]:
dallas_hpi.dtypes

DATE     object
HPI     float64
dtype: object

In [28]:
def stat_cal(df,var,by=None):
    if by == None:
        df_stat =pd.DataFrame({
            'freq': df[var].count(),          # Count of non-null values
            'min': df[var].min(),             # Minimum value
            'mean': df[var].mean(),           # Mean
            'p1': df[var].quantile(0.01),     # 1st percentile
            'p10': df[var].quantile(0.1),     # 10th percentile
            'p25': df[var].quantile(0.25),    # 25th percentile
            'median': df[var].quantile(0.5),  # Median (50th percentile)
            'p75': df[var].quantile(0.75),    # 75th percentile
            'p90': df[var].quantile(0.9),     # 90th percentile
            'p99': df[var].quantile(0.99),    # 99th percentile
            'max': df[var].max()              # Maximum value
            }, index=[0])
    else:
        df_stat =df.groupby(by)[var].agg( [
                ('freq', 'count'),
                ('min', 'min'),
                ('mean','mean'),
                ('p1', lambda x: x.quantile(0.01)),
                ('p10', lambda x: x.quantile(0.1)),
                ('p25', lambda x: x.quantile(0.25)),
                ('median', lambda x: x.quantile(0.5)),
                ('p75', lambda x: x.quantile(0.75)),
                ('p90', lambda x: x.quantile(0.9)),
                ('p99', lambda x: x.quantile(0.99)),
                ('max','max')
            ]).reset_index()

    return df_stat

In [12]:
price_stat=stat_cal(house_data, 'PRICE')
price_stat

,freq,min,mean,p1,p10,p25,median,p75,p90,p99,max
0,4308,189900,651224.801068,289900.0,374963.0,444967.5,575000.0,755004.5,975000.0,1848249.07,5495000


In [13]:
lotsize_stat= stat_cal(house_data,'LOTSIZE')
lotsize_stat

,freq,min,mean,p1,p10,p25,median,p75,p90,p99,max
0,4269,1.0,9042.521434,2614.0,5227.0,6434.0,7841.0,9235.0,11718.0,20008.08,1227956.0


In [14]:
squarefeet_stat= stat_cal(house_data,'SQUAREFEET')
squarefeet_stat

,freq,min,mean,p1,p10,p25,median,p75,p90,p99,max
0,4306,0.0,2989.338365,1303.05,1733.0,2103.25,2857.0,3734.5,4378.5,6086.9,12222.0


In [15]:
yearbulit_stat= stat_cal(house_data,'YEARBUILT')
yearbulit_stat

,freq,min,mean,p1,p10,p25,median,p75,p90,p99,max
0,4305,1902.0,2005.801858,1973.0,1992.0,2000.0,2005.0,2016.0,2021.0,2022.0,2023.0


In [16]:
beds_frequency = house_data['BEDS'].value_counts(dropna=False).reset_index().sort_values('BEDS')
print(beds_frequency)

   BEDS  count
6   1.0      2
3   2.0    172
1   3.0   1313
0   4.0   1748
2   5.0   1015
4   6.0     54
5   7.0      2
7   NaN      2


In [17]:
baths_frequency= house_data['BATHS'].value_counts(dropna=False).reset_index().sort_values('BATHS')
baths_frequency

,BATHS,count
11,1.0,7
10,1.5,7
0,2.0,1223
2,2.5,676
4,3.0,472
1,3.5,789
3,4.0,519
5,4.5,353
7,5.0,89
6,5.5,122


In [18]:
missing_in_price = house_data['PRICE'].isnull().sum()
print(missing_in_price)

0


In [12]:
dallas_hpi=dallas_hpi.sort_values(by='DATE')

In [13]:
dallas_hpi['hpi_1month_lag'] = dallas_hpi['HPI'].shift(1)  # Lag by 1 month
dallas_hpi['hpi_3month_lag'] = dallas_hpi['HPI'].shift(3)  # Lag by 3 month
dallas_hpi['hpi_6month_lag'] = dallas_hpi['HPI'].shift(6)  # Lag by 6 periods

In [14]:
dallas_hpi['hpi_1m_pct'] = (dallas_hpi['HPI']/dallas_hpi['hpi_1month_lag']-1)
dallas_hpi['hpi_3m_pct'] = (dallas_hpi['HPI']/dallas_hpi['hpi_3month_lag']-1)
dallas_hpi['hpi_6m_pct'] = (dallas_hpi['HPI']/dallas_hpi['hpi_6month_lag']-1)

In [15]:
dallas_hpi['hpi_1m_pct_lag'] = dallas_hpi['hpi_1m_pct'].shift(1)
dallas_hpi['hpi_3m_pct_lag'] = dallas_hpi['hpi_3m_pct'].shift(1)
dallas_hpi['hpi_6m_pct_lag'] = dallas_hpi['hpi_6m_pct'].shift(1)

In [16]:


# Calculate the moving average
dallas_hpi['hpi_3m_ma'] = dallas_hpi['HPI'].rolling(window=3).mean()

dallas_hpi['hpi_6m_ma'] = dallas_hpi['HPI'].rolling(window=6).mean()

In [17]:
dallas_hpi['hpi_3m_ma_lag'] = dallas_hpi['hpi_3m_ma'].shift(1)
dallas_hpi['hpi_6m_ma_lag'] = dallas_hpi['hpi_6m_ma'].shift(1)

In [18]:
dallas_hpi.head()

,DATE,HPI,hpi_1month_lag,hpi_3month_lag,hpi_6month_lag,hpi_1m_pct,hpi_3m_pct,hpi_6m_pct,hpi_1m_pct_lag,hpi_3m_pct_lag,hpi_6m_pct_lag,hpi_3m_ma,hpi_6m_ma,hpi_3m_ma_lag,hpi_6m_ma_lag
0,2000-01-01,100.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-02-01,101.82326,100.00000,NaN,NaN,0.018233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-03-01,102.03669,101.82326,NaN,NaN,0.002096,NaN,NaN,0.018233,NaN,NaN,101.286650,NaN,NaN,NaN
3,2000-04-01,103.20051,102.03669,100.00000,NaN,0.011406,0.032005,NaN,0.002096,NaN,NaN,102.353487,NaN,101.286650,NaN
4,2000-05-01,104.11440,103.20051,101.82326,NaN,0.008855,0.022501,NaN,0.011406,0.032005,NaN,103.117200,NaN,102.353487,NaN


In [19]:
house_hpi= pd.merge(house_data, dallas_hpi, left_on ='SOLD_MONTH', right_on='DATE', how='left')
house_hpi.head()

,SOLD_MONTH,CITY,ZIP,PRICE,BEDS,BATHS,SQUAREFEET,LOTSIZE,YEARBUILT,Price_per_sqrt,...,hpi_1m_pct,hpi_3m_pct,hpi_6m_pct,hpi_1m_pct_lag,hpi_3m_pct_lag,hpi_6m_pct_lag,hpi_3m_ma,hpi_6m_ma,hpi_3m_ma_lag,hpi_6m_ma_lag
0,2022-11-01,Carrollton,75007,375000,3.0,2.0,1821.0,8059.0,1974.0,206.0,...,-0.011119,-0.043833,-0.056616,-0.012736,-0.051550,-0.021790,290.880877,297.935473,295.273753,300.810867
1,2022-04-01,Carrollton,75007,389900,4.0,2.0,1706.0,7841.0,1979.0,229.0,...,0.031665,0.107343,0.160449,0.043016,0.093507,0.137435,287.142787,275.393042,277.540040,268.544692
2,2022-09-01,Carrollton,75007,608000,3.0,3.0,2721.0,9191.0,1986.0,223.0,...,-0.020609,-0.043366,0.022204,-0.019100,-0.013368,0.088610,300.540647,301.890153,304.990070,300.824133
3,2022-09-01,Carrollton,75007,410000,3.0,2.0,1822.0,7492.0,1984.0,225.0,...,-0.020609,-0.043366,0.022204,-0.019100,-0.013368,0.088610,300.540647,301.890153,304.990070,300.824133
4,2022-08-01,Carrollton,75007,598000,5.0,3.5,3141.0,9845.0,1989.0,190.0,...,-0.019100,-0.013368,0.088610,-0.004216,0.031378,0.142089,304.990070,300.824133,306.347980,296.745383


In [20]:
house_hpi['total_beds_baths']=house_hpi['BEDS']+house_hpi['BATHS']
house_hpi['sqft_pbb'] = house_hpi['SQUAREFEET']/house_hpi['total_beds_baths']

In [21]:
house_hpi['age'] = pd.to_datetime(house_hpi['SOLD_MONTH']).dt.year - house_hpi['YEARBUILT']



In [22]:
house_hpi.head()

,SOLD_MONTH,CITY,ZIP,PRICE,BEDS,BATHS,SQUAREFEET,LOTSIZE,YEARBUILT,Price_per_sqrt,...,hpi_1m_pct_lag,hpi_3m_pct_lag,hpi_6m_pct_lag,hpi_3m_ma,hpi_6m_ma,hpi_3m_ma_lag,hpi_6m_ma_lag,total_beds_baths,sqft_pbb,age
0,2022-11-01,Carrollton,75007,375000,3.0,2.0,1821.0,8059.0,1974.0,206.0,...,-0.012736,-0.051550,-0.021790,290.880877,297.935473,295.273753,300.810867,5.0,364.200000,48.0
1,2022-04-01,Carrollton,75007,389900,4.0,2.0,1706.0,7841.0,1979.0,229.0,...,0.043016,0.093507,0.137435,287.142787,275.393042,277.540040,268.544692,6.0,284.333333,43.0
2,2022-09-01,Carrollton,75007,608000,3.0,3.0,2721.0,9191.0,1986.0,223.0,...,-0.019100,-0.013368,0.088610,300.540647,301.890153,304.990070,300.824133,6.0,453.500000,36.0
3,2022-09-01,Carrollton,75007,410000,3.0,2.0,1822.0,7492.0,1984.0,225.0,...,-0.019100,-0.013368,0.088610,300.540647,301.890153,304.990070,300.824133,5.0,364.400000,38.0
4,2022-08-01,Carrollton,75007,598000,5.0,3.5,3141.0,9845.0,1989.0,190.0,...,-0.004216,0.031378,0.142089,304.990070,300.824133,306.347980,296.745383,8.5,369.529412,33.0


In [23]:
missing_in_age = house_hpi['age'].isnull().sum()
print(missing_in_age)

3


In [24]:
house_hpi[house_hpi['age'].isnull()]

,SOLD_MONTH,CITY,ZIP,PRICE,BEDS,BATHS,SQUAREFEET,LOTSIZE,YEARBUILT,Price_per_sqrt,...,hpi_1m_pct_lag,hpi_3m_pct_lag,hpi_6m_pct_lag,hpi_3m_ma,hpi_6m_ma,hpi_3m_ma_lag,hpi_6m_ma_lag,total_beds_baths,sqft_pbb,age
3240,2021-05-01,Frisco,75034,5495000,6.0,9.0,10780.0,450236.0,NaN,510.0,...,0.028948,0.075726,0.10352,226.75108,219.086308,220.516827,214.77596,15.0,718.666667,NaN
3903,2023-04-01,Frisco,75033,1195000,NaN,NaN,NaN,26049.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3974,2023-05-01,Frisco,75034,1928234,NaN,NaN,NaN,87643.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
house_hpi.dtypes

SOLD_MONTH           object
CITY                 object
ZIP                   int64
PRICE                 int64
BEDS                float64
BATHS               float64
SQUAREFEET          float64
LOTSIZE             float64
YEARBUILT           float64
Price_per_sqrt      float64
MLS                   int64
LATITUDE            float64
LONGITUDE           float64
HOUSE_DES            object
DATE                 object
HPI                 float64
hpi_1month_lag      float64
hpi_3month_lag      float64
hpi_6month_lag      float64
hpi_1m_pct          float64
hpi_3m_pct          float64
hpi_6m_pct          float64
hpi_1m_pct_lag      float64
hpi_3m_pct_lag      float64
hpi_6m_pct_lag      float64
hpi_3m_ma           float64
hpi_6m_ma           float64
hpi_3m_ma_lag       float64
hpi_6m_ma_lag       float64
total_beds_baths    float64
sqft_pbb            float64
age                 float64
dtype: object

In [26]:
house_hpi[house_hpi['age']<0]

,SOLD_MONTH,CITY,ZIP,PRICE,BEDS,BATHS,SQUAREFEET,LOTSIZE,YEARBUILT,Price_per_sqrt,...,hpi_1m_pct_lag,hpi_3m_pct_lag,hpi_6m_pct_lag,hpi_3m_ma,hpi_6m_ma,hpi_3m_ma_lag,hpi_6m_ma_lag,total_beds_baths,sqft_pbb,age


In [29]:
age_stat=stat_cal(house_hpi, 'age')
age_stat

,freq,min,mean,p1,p10,p25,median,p75,p90,p99,max
0,4305,0.0,16.043438,0.0,1.0,6.0,16.0,22.0,30.0,49.0,119.0


In [34]:
total_beds_baths_stat=stat_cal(house_hpi, 'total_beds_baths')
total_beds_baths_stat

,freq,min,mean,p1,p10,p25,median,p75,p90,p99,max
0,4306,2.0,6.993497,4.0,5.0,5.5,7.0,8.5,9.5,11.0,16.0


In [35]:
sqft_perbb__stat=stat_cal(house_hpi, 'sqft_pbb')
sqft_perbb__stat

,freq,min,mean,p1,p10,p25,median,p75,p90,p99,max
0,4306,0.0,420.163628,265.411818,330.076923,369.244505,415.0,463.97619,517.183333,627.852391,862.5


In [30]:
house_hpi_imp=house_hpi.fillna(0)

In [31]:
X = house_hpi_imp.drop(columns=['SOLD_MONTH','CITY','ZIP','PRICE','Price_per_sqrt','MLS','LATITUDE','LONGITUDE','HOUSE_DES','DATE','HPI'], axis=1)
y = house_hpi_imp['PRICE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
X.dtypes

BEDS                float64
BATHS               float64
SQUAREFEET          float64
LOTSIZE             float64
YEARBUILT           float64
hpi_1month_lag      float64
hpi_3month_lag      float64
hpi_6month_lag      float64
hpi_1m_pct          float64
hpi_3m_pct          float64
hpi_6m_pct          float64
hpi_1m_pct_lag      float64
hpi_3m_pct_lag      float64
hpi_6m_pct_lag      float64
hpi_3m_ma           float64
hpi_6m_ma           float64
hpi_3m_ma_lag       float64
hpi_6m_ma_lag       float64
total_beds_baths    float64
sqft_pbb            float64
age                 float64
dtype: object

In [39]:
model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42)
model.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.05, max_depth=6, n_estimators=200,
                          random_state=42)

In [40]:
y_pred = model.predict(X_test)

In [41]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 13019345787.368229


In [42]:
df=pd.DataFrame({"actual":y_test, "predit":y_pred})

In [43]:
df.actual.max()

3225000

In [44]:
df.predit.max()

3507657.5580288903

In [35]:
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'learning_rate': [0.05, 0.1, 0.15],
#     'max_depth': [3, 4, 5, 6, 7, 8],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [5,10],
#     'subsample': [0.6, 0.8, 1.0]
# }

param_grid = {
    'n_estimators': [ 300],
    'learning_rate': [0.1, 0.15],
    'max_depth': [3, 4, 5],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [5],
    'subsample': [0.8]
}

In [36]:
model = GradientBoostingRegressor(random_state=42)

In [37]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the final model with the best parameters
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the final model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error with best model:", mse)

Best parameters found:  {'learning_rate': 0.15, 'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 300, 'subsample': 0.8}
Mean Squared Error with best model: 11102694311.421192


In [38]:
with open(file_path +'best_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

print("Best model saved to best_model.pkl")

# Load the model from the pickle file
with open(file_path +'best_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

Best model saved to best_model.pkl


In [39]:
joblib.dump(best_model, file_path + 'model_best.pkl')

['/content/drive/MyDrive/My_project/house_price_prediction/model_best.pkl']